# Calling Python from Julia

Julia supports calling Python code via the package [PythonCall.jl](https://juliapy.github.io/PythonCall.jl/stable/). From the website:

- Call Python code from Julia and Julia code from Python via a symmetric interface.
- Simple syntax, so the Python code looks like Python and the Julia code looks like Julia.
- Intuitive and flexible conversions between Julia and Python: anything can be converted, you are in control.
- Fast non-copying conversion of numeric arrays in either direction: modify Python arrays (e.g. bytes, array.array, numpy.ndarray) from Julia or Julia arrays from Python.
- Helpful wrappers: interpret Python sequences, dictionaries, arrays, dataframes and IO streams as their Julia counterparts, and vice versa.
- Beautiful stack-traces.
- Works anywhere: tested on Windows, MacOS and Linux, 32- and 64-bit, Julia Julia 1.6.1 upwards and Python 3.8 upwards.

By default, installing `PythonCall.jl` also installs a minimal conda environment within your current active Julia environment, allowing you to keep track of the python packages you use for this project. However, you can also specify which python interpreter you want to use, if you already have python installed and want to use a specific interpreter. You can do this by setting the following environment variables (within Julia, and before import PythonCall.)

#### If you already have Python and required Python packages installed
```julia
ENV["JULIA_CONDAPKG_BACKEND"] = "Null"
ENV["JULIA_PYTHONCALL_EXE"] = "/path/to/python" 
```

#### If you already have a Conda environment
```julia
ENV["JULIA_CONDAPKG_BACKEND"] = "Current"
ENV["JULIA_CONDAPKG_EXE"] = "/path/to/conda"  
```

## Installing Python packages

Assuming you haven't opted out, PythonCall uses CondaPkg.jl to automatically install any required Python packages.

In the terminal (REPL), this is as simple as
```julia
julia> using CondaPkg

julia> # press ] to enter the Pkg REPL

pkg> conda add some_package
```

When working in a notebook, you can do

To install python packages, you can use the `CondaPkg` package.

When using a notebook, you can do:

```julia
using CondaPkg
CondaPkg.add("numpy")
```

This creates a CondaPkg.toml file in the active project specifying the dependencies, just like a Project.toml specifies Julia dependencies. Commit this file along with the rest of the project so that dependencies are automatically installed for everyone using it.

To add dependencies to a Julia package, just ensure the package project is activated first.

See the [CondaPkg.jl](https://github.com/JuliaPy/CondaPkg.jl) documentation.

## Using PythonCall

All Python objects defined using `PythonCall` are `Py` types, which support various attributes such as attribute access, function calls, indexing, comparison, and arithmetic.

In [30]:
using PythonCall

In [31]:
list = pylist([1, 2, 3])

Python: [1, 2, 3]

In [32]:
typeof(list)

Py

In [33]:
list[0] # python lists/arrays use 0-based indexing 

Python: 1

In [34]:
list.append(4)
list

Python: [1, 2, 3, 4]

In [35]:
sum(list)

Python: 10

Here, we created a Python list using the `pylist` function. Many Python function can be accessed with the `py` prefix, for example

In [36]:
for k in pyrange(5)
    println(k)
end

0
1
2
3
4


In [37]:
pydir(list)

Python: ['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']

In [38]:
pyany(pylist([0, 1, 0]))

true

Since the list we made and its content is of type `Py`, comparisons with equivalent Julia object won't work.

In [39]:
list == [1, 2, 3, 4]

false

In [40]:
list .== [1, 2, 3, 4] # element-wise comparison

4-element Vector{Py}:
 False
 False
 False
 False

To compare these two objects, we can convert the list to a Julia vector using the `pyconvert` function.

In [41]:
julia_vec = pyconvert(Vector, list)
@show julia_vec == [1, 2, 3, 4]
@show all(julia_vec .== [1, 2, 3, 4]);

julia_vec == [1, 2, 3, 4] = true
all(julia_vec .== [1, 2, 3, 4]) = true


`pyconvert` is not just used for converting Python lists to Julia vectors, but many other Python types to the equivalent types in Julia

In [46]:
python_words = pylist(["Hello", "world!"])
python_sentence = Py(" ").join(python_words)

Python: 'Hello world!'

In [47]:
pyconvert(String, python_sentence) # convert to Julia string

"Hello world!"

## Using Python packages

To import a Python package into your session, use the `pyimport` function and assign the output to a variable. For example:

In [44]:
python_math = pyimport("math")

python_math.sqrt(list[-1])

Python: 2.0

To do something equivalent to the python statemement `from X import Y as Z`, you can call `pyimport` using the signature `Z = pyimport(X => Y)`. For example:

In [45]:
math_pi = pyimport("math" => "pi")
math_pi

Python: 3.141592653589793

In [50]:
pyconvert(Float64, math_pi)

3.141592653589793